In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import os

# 1. === Configuration ===
input_files = {
    "2024-07-12": "/content/WSE_nodes_2024-07-12.xlsx",
    "2024-08-13": "/content/WSE_nodes_2024-08-13.xlsx",
    "2024-09-08": "/content/WSE_nodes_2024-09-08.xlsx"
}

x_break = 514305  # UTM coordinate marking the landslide initiation point (used to separate upstream and downstream segments)

# Storage for results
results = []

# 2. === Function to compute slope statistics ===
def compute_slope_stats(df, segment_name, date):
    x = df['X_UTM']
    y = df['WSE']
    n = len(df)

    if n < 3:
        return {
            "Date": date,
            "Segment": segment_name,
            "Slope (m/km)": np.nan,
            "Std Error (m/km)": np.nan,
            "95% CI Lower (m/km)": np.nan,
            "95% CI Upper (m/km)": np.nan,
            "P-value": np.nan
        }

    slope, intercept = np.polyfit(x, y, deg=1)
    residuals = y - (slope * x + intercept)

    stderr = np.sqrt(np.sum(residuals**2) / (n - 2)) / np.sqrt(np.sum((x - np.mean(x))**2))
    t_val = stats.t.ppf(0.975, n - 2)
    ci_low = (slope - t_val * stderr) * 1000
    ci_high = (slope + t_val * stderr) * 1000
    p_val = 2 * (1 - stats.t.cdf(abs(slope / stderr), n - 2))

    return {
        "Date": date,
        "Segment": segment_name,
        "Slope (m/km)": abs(slope * 1000),
        "Std Error (m/km)": stderr * 1000,
        "95% CI Lower (m/km)": abs(ci_low),
        "95% CI Upper (m/km)": abs(ci_high),
        "P-value": p_val
    }


# 3. === Main loop ===
for date, filepath in input_files.items():
    if not os.path.exists(filepath):
        print(f"File not found for {date}: {filepath}")
        continue

    df = pd.read_excel(filepath)

    # Keep only node data with valid WSE
    nodes = df[(df['data_type'] == 'nodes') & (df['WSE'].notna())]

    upstream = nodes[nodes['X_UTM'] < x_break].copy()
    downstream = nodes[nodes['X_UTM'] >= x_break].copy()

    # Compute stats
    res_up = compute_slope_stats(upstream, "Upstream", date)
    res_down = compute_slope_stats(downstream, "Downstream", date)

    results.extend([res_up, res_down])

# 4. === Export results as DataFrame ===
results_df = pd.DataFrame(results)

# Display
print("\n=== Slope Statistics (m/km) ===")
print(results_df.to_string(index=False))



=== Slope Statistics (m/km) ===
      Date    Segment  Slope (m/km)  Std Error (m/km)  95% CI Lower (m/km)  95% CI Upper (m/km)      P-value
2024-07-12   Upstream      3.140296          0.098580             3.357269             2.923323 3.469669e-12
2024-07-12 Downstream      2.951194          0.050917             3.056789             2.845599 0.000000e+00
2024-08-13   Upstream      0.017865          0.010234             0.039975             0.004244 1.044465e-01
2024-08-13 Downstream      4.813136          0.185838             5.200787             4.425485 0.000000e+00
2024-09-08   Upstream      0.592258          0.186792             0.995797             0.188719 7.372118e-03
2024-09-08 Downstream      4.398707          0.139988             4.689827             4.107587 0.000000e+00
